In [ ]:
from collections import namedtuple, OrderedDict
from functools import reduce

class StockTradeDays(object): 
    
    def __init__(self, 
                 price_array, 
                 start_date, 
                 date_array=None):
        # Private Price Array
        self.__price_array = price_array
        # Private Date Array
        self.__date_array = self._init_days(start_date, date_array)
        # Private Change Array
        self.__change_array = self.__init_change()
        # Construct OrderedDict
        self.stock_dict = self._init_stock_dict()
    
    def __init_change(self):
        """
        Generate change_array from price_array
        :return: 
        """
        price_float_array = [float(price_str) for price_str in self.__price_array]
        
        # Move the price to generate a change_array
        pp_array = [(price1, price2) for price1, price2 in zip(price_float_array[:-1], price_float_array[1:])]
        change_array = map(
            lambda pp: reduce(
                lambda  a, b: round(
                    (b - a) / a, 
                    3), 
                pp),
            pp_array)
        
        # list insert() function insert first date change as 9. 
        change_array.insert(0, 0)
        return change_array
    
    def _init_days(self, start_date, date_array):
        """
        protected function,
        :param start_date: First date
        :param date_array: date_array
        :return:
        """
        if date_array is None:
            # Use start_date and self.__price_arary to determine date_array
            date_array = [str(start_date + index) for index, _ in enumerate(self.__price_array)]
        else:
            # Use date_array if provided
            date_array = [str(date) for date in date_array]
        return date_array
    
    def _init_stock_dic(self):
        """
        Use namedtupple, OrderedDict to combine results
        :return: 
        """
        stock_namedtuple = namedtuple('stock', 
                                      ('date', 'price', 'change'))
        # Use defined __date_array to combine OrderedDict
        stock_dict = OrderedDict(
            (date, stock_namedtuple(date, price, change)
             for date, price, change in
             zip (self.__date_array, self.__price_array, self.__change_array))
        )
        return stock_dict
    
    def filter_stock(self, up_days_only=True, want_calc_sum=False):
        """
        Filter stock result
        :param up_days_only: 
        :param want_calc_sum: 
        :return: 
        """
        # Use filter_func as Filter
        filter_func = (lambda day: day.change > 0) if up_days_only \
            else (lambda day: day.change < 0)
        
        filtered_days = filter(filter_func, self.stock_dict.values())
        
        if not want_calc_sum:
            return filtered_days
        
        # Need to calculate the sum
        change_sum = 0.0
        for day in want_days:
            change_sum += day.change
            return change_sum
        
    def __str__(self):
        return str(self.stock_dict)
    
    __repr__ = __str__
        
    def __iter__(self):
        """
        Iterate stock_dict to yield elements
        :return: 
        """
        for key in self.stock_dict:
            yield self.stock_dict[key]
        
    def __getitem__(self, index):
        date_key = self.__date_array[index]
        return self.stock_dict[date_key]
    
    def __len__(self):
        return len(self.stock_dict)
        